In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import config

In [2]:
def parse_jssp_file(filename):
    jobs_data = []

    with open(filename, "r") as f:
        # Enlever les lignes de commentaires
        lines = [
            line.strip()
            for line in f
            if line.strip() and not line.strip().startswith("#")
        ]

    # Première ligne : nb jobs, nb machines
    num_jobs, num_machines = map(int, lines[0].split())

    # Lignes suivantes : jobs
    for i in range(1, num_jobs + 1):
        data = list(map(int, lines[i].split()))
        job = []

        for k in range(0, len(data), 2):
            machine = data[k]
            duration = data[k + 1]
            job.append((machine, duration))

        jobs_data.append(job)

    return jobs_data, num_jobs, num_machines

jobs_data, num_jobs, num_machines = parse_jssp_file(config.INSTANCE_DIR + "/abz5.txt")


In [7]:
from ortools.linear_solver import pywraplp

def solve_with_nogood_learning(jobs_data, num_jobs, num_machines, max_iterations=5, time_limit_per_iter=60):
    print(f"--- Approche NoGood Learning / Solution Elimination (SCIP) ---")

    # Création du solveur
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return

    # --- 1. Modélisation initiale (Identique au MILP classique) ---
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Variables de début
    starts = {}
    for j in range(num_jobs):
        for t in range(num_machines):
            starts[(j, t)] = solver.IntVar(0.0, float(horizon), f'start_{j}_{t}')

    makespan = solver.IntVar(0.0, float(horizon), 'makespan')

    # Précédence intra-job
    for j in range(num_jobs):
        for t in range(len(jobs_data[j]) - 1):
            dur = jobs_data[j][t][1]
            solver.Add(starts[(j, t + 1)] >= starts[(j, t)] + dur)
        # Makespan
        last_t = len(jobs_data[j]) - 1
        last_dur = jobs_data[j][last_t][1]
        solver.Add(makespan >= starts[(j, last_t)] + last_dur)

    # Variables de Disjonction (Binaires) - C'est ce qu'on va "Apprendre"
    # disj_vars[(m, j1, j2)] stockera la variable binaire
    disj_vars = {}

    task_on_machine = {m: [] for m in range(num_machines)}
    for j in range(num_jobs):
        for t, (mach, dur) in enumerate(jobs_data[j]):
            task_on_machine[mach].append((j, t, dur))

    M = float(horizon)

    for m in range(num_machines):
        tasks = task_on_machine[m]
        for idx1 in range(len(tasks)):
            for idx2 in range(idx1 + 1, len(tasks)):
                j1, t1, dur1 = tasks[idx1]
                j2, t2, dur2 = tasks[idx2]

                # Variable binaire : 1 si j1 avant j2, 0 sinon
                y = solver.IntVar(0, 1, f'y_m{m}_{j1}_{j2}')

                # Sauvegarde pour générer les NoGoods plus tard
                disj_vars[(m, j1, j2)] = y

                # Contraintes Big-M
                solver.Add(starts[(j2, t2)] >= starts[(j1, t1)] + dur1 - M * (1 - y))
                solver.Add(starts[(j1, t1)] >= starts[(j2, t2)] + dur2 - M * y)

    solver.Minimize(makespan)

    # --- 2. Boucle d'Apprentissage (Learning Loop) ---

    best_makespan = float('inf')

    for iteration in range(max_iterations):
        print(f"\n--- Itération {iteration + 1}/{max_iterations} ---")

        # Si on a déjà trouvé une solution, on peut contraindre le makespan
        # pour forcer l'amélioration (Technique Branch & Cut)
        if best_makespan < float('inf'):
            # "NoGood" sur la qualité : On ne veut plus de solutions pires que la meilleure actuelle
            # On ajoute une contrainte : makespan <= best_makespan - 1
            solver.Add(makespan <= best_makespan - 1)
            print(f"Ajout contrainte d'amélioration : Makespan <= {best_makespan - 1}")

        # Paramètres
        solver.SetTimeLimit(time_limit_per_iter * 1000)

        status = solver.Solve()

        if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
            current_makespan = solver.Objective().Value()
            print(f"Solution trouvée ! Makespan: {current_makespan}")

            if current_makespan < best_makespan:
                best_makespan = current_makespan

            # --- GÉNÉRATION DU NOGOOD (Canonical Cut) ---
            # On récupère la configuration binaire exacte de cette solution
            # Et on crée une contrainte qui l'interdit pour le futur.
            # Formule du "Integer Cut" :
            # Somme(vars qui étaient à 1) - Somme(vars qui étaient à 0) <= Nombre_de_1 - 1

            vars_at_1 = []
            vars_at_0 = []

            for key, var in disj_vars.items():
                val = var.solution_value()
                if val > 0.5: # C'est un 1
                    vars_at_1.append(var)
                else:         # C'est un 0
                    vars_at_0.append(var)

            n_ones = len(vars_at_1)

            print(f"Apprentissage du NoGood (Interdiction de la configuration actuelle)...")
            # Le NoGood cut : Empêche cette combinaison exacte de se reproduire
            # sum(y for y in ones) - sum(y for y in zeros) <= |ones| - 1

            cut_expr = solver.Sum(vars_at_1) - solver.Sum(vars_at_0)
            solver.Add(cut_expr <= n_ones - 1)

            print(f"Cut ajouté. (Taille du cut: {len(vars_at_1)} variables positives)")

        else:
            print("Aucune nouvelle solution trouvée (Infeasible ou TimeLimit).")
            # Si c'est Infeasible avec la contrainte d'amélioration,
            # ça veut dire qu'on a probablement trouvé l'optimum avant.
            break

    print(f"\nTerminé. Meilleur Makespan global : {best_makespan}")

# --- Test ---
jobs_data, num_jobs, num_machines = parse_jssp_file("../instances/ft10.txt")
solve_with_nogood_learning(jobs_data, num_jobs, num_machines)

--- Approche NoGood Learning / Solution Elimination (SCIP) ---

--- Itération 1/5 ---
Solution trouvée ! Makespan: 949.0
Apprentissage du NoGood (Interdiction de la configuration actuelle)...
Cut ajouté. (Taille du cut: 234 variables positives)

--- Itération 2/5 ---
Ajout contrainte d'amélioration : Makespan <= 948.0
Solution trouvée ! Makespan: 947.0
Apprentissage du NoGood (Interdiction de la configuration actuelle)...
Cut ajouté. (Taille du cut: 198 variables positives)

--- Itération 3/5 ---
Ajout contrainte d'amélioration : Makespan <= 946.0
Solution trouvée ! Makespan: 937.0
Apprentissage du NoGood (Interdiction de la configuration actuelle)...
Cut ajouté. (Taille du cut: 168 variables positives)

--- Itération 4/5 ---
Ajout contrainte d'amélioration : Makespan <= 936.0
Aucune nouvelle solution trouvée (Infeasible ou TimeLimit).

Terminé. Meilleur Makespan global : 937.0


In [6]:
from ortools.sat.python import cp_model
import random

def solve_subproblem(jobs_data, num_jobs, num_machines, current_starts, best_makespan, relaxation_rate=0.3, time_limit=0.5):
    """
    Crée et résout un sous-problème où une partie des tâches sont FIXÉES
    à leur valeur actuelle, et d'autres sont RELACHÉES pour être optimisées.
    """
    model = cp_model.CpModel()
    horizon = int(best_makespan * 1.5) # Horizon réduit pour le sous-problème

    # Structures
    named_tasks = {}
    machine_to_intervals = {m: [] for m in range(num_machines)}
    job_ends = []

    # --- 1. Reconstruction du modèle ---
    # La clé de la LNS est ici : On décide quelles tâches on fige ou on libère

    # On choisit aléatoirement des Jobs à relacher (ou on pourrait choisir une fenêtre de temps)
    # Ici : chaque job a 'relaxation_rate' chance d'être relâché (re-optimisé)
    jobs_to_relax = set()
    for j in range(num_jobs):
        if random.random() < relaxation_rate:
            jobs_to_relax.add(j)

    # Création des variables
    for j in range(num_jobs):
        for t_idx, (machine, duration) in enumerate(jobs_data[j]):
            suffix = f'_{j}_{t_idx}'

            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)

            named_tasks[(j, t_idx)] = {'start': start_var, 'end': end_var, 'interval': interval_var}
            machine_to_intervals[machine].append(interval_var)

            # --- C'EST ICI QUE L'APPRENTISSAGE SE FAIT ---
            # Si le job n'est PAS dans la liste à relâcher, on le FIGE à sa valeur actuelle.
            # Le solveur n'a pas besoin de réfléchir pour ces tâches-là.
            if j not in jobs_to_relax and (j, t_idx) in current_starts:
                model.Add(start_var == current_starts[(j, t_idx)])

    # Contraintes classiques (Précédence & NoOverlap)
    for j in range(num_jobs):
        for t_idx in range(len(jobs_data[j]) - 1):
            model.Add(named_tasks[(j, t_idx + 1)]['start'] >= named_tasks[(j, t_idx)]['end'])

    for m in range(num_machines):
        model.AddNoOverlap(machine_to_intervals[m])

    # Objectif : Makespan
    for j in range(num_jobs):
        last_t = len(jobs_data[j]) - 1
        job_ends.append(named_tasks[(j, last_t)]['end'])

    makespan = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(makespan, job_ends)

    # Contrainte d'amélioration stricte : On veut MIEUX que le best actuel
    model.Add(makespan < best_makespan)
    model.Minimize(makespan)

    # Résolution rapide du sous-problème
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    solver.parameters.num_search_workers = 1 # Pas besoin de multi-thread pour un sous-problème

    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        new_starts = {}
        for j in range(num_jobs):
            for t_idx in range(len(jobs_data[j])):
                new_starts[(j, t_idx)] = solver.Value(named_tasks[(j, t_idx)]['start'])
        return new_starts, solver.ObjectiveValue()

    return None, None

def solve_jssp_lns_integrated(jobs_data, num_jobs, num_machines, total_time_limit=60):
    print(f"--- Lancement de l'approche LNS (Large Neighborhood Search) ---")

    # 1. Solution Initiale (On lance un CP-SAT standard court pour avoir une base)
    print("Recherche d'une solution initiale...")
    # (J'utilise ici une version simplifiée inline pour l'initialisation)
    # Dans ton code réel, appelle ta fonction solve_cpsat_jsp avec un timer court (ex: 2s)
    # Supposons qu'on ait une solution 'current_starts' et un 'current_makespan'
    # Pour l'exemple, imaginons qu'on l'obtienne via une heuristique ou un premier run :

    # --- Code CP-SAT Standard (Initialisation) ---
    model = cp_model.CpModel()
    horizon = sum(task[1] for job in jobs_data for task in job)
    # ... (Déclaration variables et contraintes standard identiques au code précédent) ...
    # (Je simplifie ici pour la lisibilité, on suppose qu'on a la logique standard)
    # [Insérer ici la logique de création du modèle initial]
    # ...
    # Disons qu'on a fait un premier run :
    # current_starts = ...
    # current_makespan = ...

    # NOTE: Pour que ce code fonctionne directement, remplace ce bloc par un appel
    # à ta fonction solve_cpsat_jsp(..., time_limit=2) :
    current_starts, current_makespan = solve_cpsat_jsp(jobs_data, num_jobs, num_machines, time_limit_sec=2)

    if not current_starts:
        print("Impossible de trouver une solution initiale.")
        return

    print(f"Solution initiale: {current_makespan}")

    # 2. Boucle LNS
    import time
    start_time = time.time()
    iteration = 0

    while time.time() - start_time < total_time_limit:
        iteration += 1
        remaining_time = total_time_limit - (time.time() - start_time)

        # On tente d'améliorer
        # On relaxe 30% des jobs
        new_starts, new_makespan = solve_subproblem(
            jobs_data, num_jobs, num_machines,
            current_starts, current_makespan,
            relaxation_rate=0.3, # 30% de chaos
            time_limit=0.5       # Résolution très rapide (0.5s)
        )

        if new_starts:
            print(f"Iter {iteration}: Amélioration ! {current_makespan} -> {new_makespan}")
            current_starts = new_starts
            current_makespan = new_makespan
        else:
            # Si on ne trouve pas, on continue juste la boucle,
            # le random choisira d'autres jobs à relaxer au prochain tour.
            pass

    print(f"\nTerminé. Meilleur Makespan LNS : {current_makespan}")
    return current_starts

# --- Note d'usage ---
# Ce code nécessite ta fonction 'solve_cpsat_jsp' définie précédemment pour l'initialisation.
jobs_data, num_jobs, num_machines = parse_jssp_file("../instances/abz5.txt")
solve_jssp_lns_integrated(jobs_data, num_jobs, num_machines)

--- Lancement de l'approche LNS (Large Neighborhood Search) ---
Recherche d'une solution initiale...


ModuleNotFoundError: No module named 'main_cpsat'